In [29]:
# # Fix the PHESANT output for regenie
# Again, more wrangling. For ease of use and further analysis, dataframes must contain exlusively quantitative or binary columns—not a mix

import pandas as pd
import numpy as np
import subprocess

# Set FLAGS

PHENOTYPE = "metabolic"
TRAIT = "BT"

In [2]:
# Load output from PHESANT and fix headers

file = f"/mnt/project/Data/phenotypes/{PHENOTYPE}.{TRAIT}.phesant/{PHENOTYPE}.phesantdata-binary-all.txt"

df = pd.read_csv(file, sep=",", index_col="userID")

df.columns = [s.replace("#", "_") for s in df.columns]

for col in df.columns:

    assert df.loc[:, col].nunique() == 2, "Uh oh, column is not binary"

    df.loc[:, col] = df.loc[:, col].astype("Int64")

    print(col, "\n", df.loc[:, col].value_counts(), "\n", sep="")

2443
0    473399
1     26395
Name: 2443, dtype: Int64

2463
0    451150
1     47455
Name: 2463, dtype: Int64

130706
0    497228
1      5181
Name: 130706, dtype: Int64

130708
0    458577
1     43832
Name: 130708, dtype: Int64

130792
0    453560
1     48849
Name: 130792, dtype: Int64

131674
0    468573
1     33836
Name: 131674, dtype: Int64

131676
0    495900
1      6509
Name: 131676, dtype: Int64

131688
0    496994
1      5415
Name: 131688, dtype: Int64

131690
0    498874
1      3535
Name: 131690, dtype: Int64



In [ ]:
file = f"/mnt/project/Data/phenotypes/{PHENOTYPE}.{TRAIT}.phesant/{PHENOTYPE}.phesantdata-catord-all.txt"

cat = pd.read_csv(file, sep=",", index_col="userID")

cat.columns = [s.replace("#", "_") for s in cat.columns]

In [39]:
w = pd.get_dummies(cat.loc[:,"1687"], prefix = "1687")
w.loc[w.eq(0).all(axis = 1),:] = np.nan
w.columns = [s.replace(".0", "") for s in w.columns]
w = w.astype("Int64")

h = pd.get_dummies(cat.loc[:,"1697"], prefix = "1697")
h.loc[h.eq(0).all(axis = 1),:] = np.nan
h.columns = [s.replace(".0", "") for s in h.columns]
h = h.astype("Int64")

w.head()

In [43]:
df = df.join(w).join(h)

In [44]:
# regenie expects these two first columns

df.insert(0, "FID", df.index)
df.insert(1, "IID", df.index)

# %%
# Save with tabs and upload

df.to_csv(
    f"../tmp/{PHENOTYPE}.{TRAIT}.final.tsv",
    sep="\t",
    na_rep="NA",
    index=False,
)


subprocess.run(
    [
        "dx",
        "upload",
        f"../tmp/{PHENOTYPE}.{TRAIT}.final.tsv",
        "--path",
        "Data/phenotypes/",
    ],
    check=True,
    shell=False,
)

CompletedProcess(args=['dx', 'upload', '../tmp/metabolic.BT.final.tsv', '--path', 'Data/phenotypes/'], returncode=0)